In [ ]:
from report import load_report, Benchmark
from typing import List
from pathlib import Path

# for plotting
import matplotlib.pyplot as plt
import seaborn as sbn
import pandas as pnd

# Load latest report

In [ ]:
current = load_report(Path('../../reports/current-09-09'))

# Print basic stats

In [ ]:
print(f'reports: {len(current)}')

functions = sorted(set((rep.function for rep in current)))
print(f'functions: {len(functions)}')

targets = sorted(set((rep.target for rep in current)))
print(f'targets: {targets}')

dimensions = sorted(set((rep.dimension for rep in current)))
print(f'dimensions: {dimensions}')

batch_sizes = sorted(set((rep.batch_size for rep in current)))
print(f'batch_sizes: {batch_sizes}')

# Create data frame from report

In [ ]:
def make_dataframe(benchmarks: List[Benchmark]) -> pnd.DataFrame:
    targets = [b.target for b in benchmarks]
    functions = [b.function for b in benchmarks]
    dimensions = [b.dimension for b in benchmarks]
    batch_sizes = [b.batch_size for b in benchmarks]
    medians = [b.measurements.estimates['median'].point for b in benchmarks]
    means = [b.measurements.estimates['mean'].point for b in benchmarks]

    return pnd.DataFrame({
        'target': targets,
        'function': functions,
        'dimension': dimensions,
        'batch_size': batch_sizes,
        'median': medians,
        'mean': means,
    })

cdf = make_dataframe(current)

In [ ]:
cdf['median_n'] = cdf['median'] / cdf.groupby(['function', 'dimension'])['median'].transform('max')

# Plot each function over batch size

In [ ]:
sbn.relplot(data=cdf, kind='line', x='batch_size', y='median_n', hue='target', row='function', col='dimension')

# Plot improvements over all functions

In [ ]:
s1 = cdf[cdf['target'] == 'sequential']['median_n']
s2 = cdf[cdf['target'] == 'multicore']['median_n']

r1 = cdf[cdf['target'] == 'reference']['median_n']
r2 = cdf[cdf['target'] == 'reference']['median_n']

r1.index = s1.index
r2.index = s2.index

d1 = s1 - r1
d2 = s2 - r2

cdf['delta'] = d1.add(d2, fill_value=0)

In [ ]:
width = 1

data = cdf[(cdf['dimension'] == 40) & (cdf['batch_size'] == 128)]
data_s = data[(data['target'] == 'sequential')]
data_m = data[(data['target'] == 'multicore')]

# Initialize the matplotlib figure
f, ax = plt.subplots(figsize=(6, 10))

# Plot the total crashes
sbn.set_color_codes("pastel")
sbn.barplot(x="delta", y="function", data=data_s, label="Sequential", errorbar=None, color="b")

# Plot the crashes where alcohol was involved
sbn.set_color_codes("muted")
sbn.barplot(x="delta", y="function", data=data_m, label="Multicore", errorbar=None, color="b")

# Add a legend and informative axis label
ax.legend(ncol=1, loc="lower right", frameon=True)
ax.set(xlim=(-width, width), ylabel="", xlabel="Relative Improvement of D40xB128")
sbn.despine(left=True, bottom=True)
